In [53]:
import pandas as pd
import numpy as np
import plotly.express as px

# simplest model

## setup the cadcad model

In [54]:

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [113]:
initial_state = {
    'impactful_proj': 250,
    'less_impactful_proj': 100,
    'total_funding':100, #any unit.
}

In [114]:
system_params = {
    'fund2proj':[0.1], #any positive number; depends on the unit of "total_funding".
    'reward_fund2proj':[0.01,0.1], #any positive number; depends on the unit of "total_funding".
    'funding_increase':[10] #any positive or negative number (i.e. decrease); same unit as "total_funding".
}

In [115]:
# policy functions
def p_reward_fund_proj(params, subbstep, state_history, previous_state):
    value = params['reward_fund2proj'] * previous_state['total_funding']
    # Output
    return {'add_reward_proj': value}

def p_fund_proj(params, subbstep, state_history, previous_state):
    value = params['fund2proj'] * previous_state['total_funding'] 
    # Output
    return {'add_fund_proj': value}

def p_addfund(params, subbstep, state_history, previous_state):
    value = params['funding_increase'] 
    # Output
    return {'add_fund': value}
    

In [116]:
# state update functions
def s_impact_proj(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables & logic
    current_i_p = previous_state['impactful_proj']
    i_p_change = policy_input['add_reward_proj'] + policy_input['add_fund_proj']
    
    new_i_p = max(current_i_p + i_p_change, 0)
    
    # Output
    return ('impactful_proj', new_i_p)

def s_lessimpact_proj(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables
    current_li_p = previous_state['less_impactful_proj']
    li_p_change = policy_input['add_fund_proj']
    
    # Logic
    new_li_p = max(current_li_p + li_p_change, 0)
    
    # Output
    return ('less_impactful_proj', new_li_p)

def s_totalfund(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables
    current_fund = previous_state['total_funding']
    fund_change = policy_input['add_fund']
    
    # Logic
    new_fund = max(current_fund + fund_change, 0)
    
    # Output
    return ('total_funding', new_fund)

In [117]:
# state update
## serial version
"""
partial_state_update_blocks = [
    {
        'label': 'impactful project update', # Useful metadata to describe our partial state update blocks
        'policies': {
            'fund2proj': p_fund_proj,
            'reward2proj': p_reward_fund_proj,
        },
        'variables': {
            'impactful_proj': s_impact_proj
            
        }
    },
    {
        'label': 'less impactful project update', # Useful metadata to describe our partial state update blocks
        'policies': {
            'fund2proj': p_fund_proj
        },
        'variables': {
            'less_impactful_proj': s_lessimpact_proj
            
        }
    }
]"""

## parallel version
partial_state_update_blocks = [
    {
        'policies': {
            'fund2proj': p_fund_proj,
            'reward2proj': p_reward_fund_proj,
            'addfund': p_addfund 
        },
        'variables': {
            'impactful_proj': s_impact_proj,
            'less_impactful_proj': s_lessimpact_proj,
            'total_funding':s_totalfund
        }
    }
]

In [126]:
# cadcad configuration
MONTE_CARLO_RUNS = 1
SIMULATION_TIMESTEPS = 48 # time unit: week

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params,
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

In [127]:
# execute cadcad
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (48, 3, 2, 3)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 1]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized
Total execution time: 0.02s


In [128]:
df = pd.DataFrame(system_events)
# Clean substeps (removed a few of repeated rows?)
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

df

,impactful_proj,less_impactful_proj,total_funding,simulation,subset,run,timestep
0,250.0,100.0,100,0,0,1,0
1,261.0,110.0,110,0,0,1,1
2,273.1,121.0,120,0,0,1,2
3,286.3,133.0,130,0,0,1,3
4,300.6,146.0,140,0,0,1,4
...,...,...,...,...,...,...,...
93,3022.0,1486.0,540,0,1,1,44
94,3130.0,1540.0,550,0,1,1,45
95,3240.0,1595.0,560,0,1,1,46
96,3352.0,1651.0,570,0,1,1,47


In [129]:
# Attribute system parameters to each row (to contrast the simulations under different parameters)
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])

In [130]:
df.head()

,impactful_proj,less_impactful_proj,total_funding,simulation,subset,run,timestep,fund2proj,reward_fund2proj,funding_increase
0,250.0,100.0,100,0,0,1,0,0.1,0.01,10
1,261.0,110.0,110,0,0,1,1,0.1,0.01,10
2,273.1,121.0,120,0,0,1,2,0.1,0.01,10
3,286.3,133.0,130,0,0,1,3,0.1,0.01,10
4,300.6,146.0,140,0,0,1,4,0.1,0.01,10


## visualization

In [131]:
fig_df = df.query('reward_fund2proj==0.01')

fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=[fig_df.impactful_proj,fig_df.less_impactful_proj],
    #color=fig_df.co2_annual_emissions.astype(str),
    opacity=1,
    title='reward parameter = 0.01'
    #trendline="lowess",
    #labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

In [132]:
fig_df = df.query('reward_fund2proj==0.1')

fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=[fig_df.impactful_proj,fig_df.less_impactful_proj],
    #color=fig_df.co2_annual_emissions.astype(str),
    opacity=1,
    title='reward parameter = 0.1'
    #trendline="lowess",
    #labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

In [133]:
fig_df = df
fig_df['impact_ratio'] = fig_df['impactful_proj'] / fig_df['less_impactful_proj']
fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=fig_df.impact_ratio,
    color=fig_df.reward_fund2proj.astype(str),
    opacity=1,
    title='impact ratio under different reward parameters',
    #trendline="lowess",
    labels={'color': 'Reward parameter'}
)

fig.show()

Using this metrics of impact ratio (number of impactful projects divided by number of non-impactful ones), we can see that this ratio will in crease with the positive reward parameter, representing that if there's a reward system that encourages more impactful project, there will be more impactful project on the DAO platform.

Furthermore, this metrics increase is sub-linear because both impactful and unimpactful projects will be increasing sup-linearly with the assumed steady increase of funding.

# A bit more complex model

Difference: 1) the reward mechanism will drain some of the fundings 2) more impactful projects will attract more fundings

In [134]:
initial_state = {
    'impactful_proj': 250,
    'less_impactful_proj': 100,
    'total_funding':100, #any unit.
}
system_params = {
    'fund2proj':[0.1], #any positive number; depends on the unit of "total_funding".
    'reward_fund2proj':[0.2], #a ratio. any positive number; depends on the unit of "total_funding".
    'funding_increase':[10], #any positive or negative number (i.e. decrease); same unit as "total_funding".
    'reward_use_fund':[0.01,.1,0.5], #a ratio of how much existing fund will be used for granting reward
    'impactful_proj2fund':[.05,.1,.2]
}

In [135]:
# policy functions
def p_reward_fund_proj(params, subbstep, state_history, previous_state):
    value = params['reward_fund2proj'] * previous_state['total_funding']
    # Output
    return {'add_reward_proj': value}

def p_fund_proj(params, subbstep, state_history, previous_state):
    value = params['fund2proj'] * previous_state['total_funding'] 
    # Output
    return {'add_fund_proj': value}

def p_addfund(params, subbstep, state_history, previous_state):
    value = params['funding_increase'] 
    # Output
    return {'add_fund': value}
def p_rew_use_fund(params, subbstep, state_history, previous_state):
    value = params['reward_use_fund'] * previous_state['total_funding'] 
    # Output
    return {'rew_use_fund': value}    
def p_impactful_proj2fund(params, subbstep, state_history, previous_state):
    value = params['impactful_proj2fund'] * previous_state['impactful_proj'] 
    # Output
    return {'impactful_proj2fund': value}    

In [136]:
# state update functions
def s_impact_proj(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables
    current_i_p = previous_state['impactful_proj']
    i_p_change = policy_input['add_reward_proj'] + policy_input['add_fund_proj']
    
    # Logic
    new_i_p = max(current_i_p + i_p_change, 0)
    
    # Output
    return ('impactful_proj', new_i_p)

def s_lessimpact_proj(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables
    current_li_p = previous_state['less_impactful_proj']
    li_p_change = policy_input['add_fund_proj']
    
    # Logic
    new_li_p = max(current_li_p + li_p_change, 0)
    
    # Output
    return ('less_impactful_proj', new_li_p)

def s_totalfund(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables
    current_fund = previous_state['total_funding']
    fund_change = policy_input['add_fund'] - policy_input['rew_use_fund'] + policy_input['impactful_proj2fund']
    
    # Logic
    new_fund = current_fund + fund_change
    
    # Output
    return ('total_funding', new_fund)

In [137]:
# state update
## parallel version
partial_state_update_blocks = [
    {
        'policies': {
            'fund2proj': p_fund_proj,
            'reward2proj': p_reward_fund_proj,
            'addfund': p_addfund,
            'rew_use_fund':p_rew_use_fund,
            'impactful_proj2fund':p_impactful_proj2fund
        },
        'variables': {
            'impactful_proj': s_impact_proj,
            'less_impactful_proj': s_lessimpact_proj,
            'total_funding':s_totalfund
        }
    }
]

In [138]:
# cadcad configuration
MONTE_CARLO_RUNS = 1
SIMULATION_TIMESTEPS = 48 # time unit: week

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params,
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

# execute cadcad
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (48, 5, 3, 3)
Execution Method: local_simulations
SimIDs   : [0, 0, 0]
SubsetIDs: [0, 1, 2]
Ns       : [0, 1, 2]
ExpIDs   : [0, 0, 0]
Execution Mode: parallelized
Total execution time: 0.03s


In [139]:
df = pd.DataFrame(system_events)
# Clean substeps (removed a few of repeated rows?)
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])
# Attribute system parameters to each row (to contrast the simulations under different parameters)
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])
df.head()

,impactful_proj,less_impactful_proj,total_funding,simulation,subset,run,timestep,fund2proj,reward_fund2proj,funding_increase,reward_use_fund,impactful_proj2fund
0,250.000000,100.000000,100.000000,0,0,1,0,0.1,0.2,10,0.01,0.05
1,280.000000,110.000000,121.500000,0,0,1,1,0.1,0.2,10,0.01,0.05
2,316.450000,122.150000,144.285000,0,0,1,2,0.1,0.2,10,0.01,0.05
3,359.735500,136.578500,168.664650,0,0,1,3,0.1,0.2,10,0.01,0.05
4,410.334895,153.444965,194.964778,0,0,1,4,0.1,0.2,10,0.01,0.05


In [140]:
fig_df = df.query('impactful_proj2fund==0.1')

fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=[fig_df.impactful_proj,fig_df.less_impactful_proj],
    #color=fig_df.co2_annual_emissions.astype(str),
    opacity=1,
    title='impact2fund parameter = 0.1'
    #trendline="lowess",
    #labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

In [141]:
fig_df = df.query('impactful_proj2fund==0.2')

fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=[fig_df.impactful_proj,fig_df.less_impactful_proj],
    #color=fig_df.co2_annual_emissions.astype(str),
    opacity=1,
    title='impact2fund parameter = 0.2'
    #trendline="lowess",
    #labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

In [143]:
fig_df = df
fig_df['impact_ratio'] = fig_df['impactful_proj'] / fig_df['less_impactful_proj']
fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=fig_df.impact_ratio,
    color=fig_df.impactful_proj2fund.astype(str),
    opacity=1,
    title='impact ratio under different impactful funding2fund parameters',
    #trendline="lowess",
    labels={'color': 'Impact parameter'}
)

fig.show()

In [110]:
fig_df = df
fig_df['impact_ratio'] = fig_df['impactful_proj'] / fig_df['less_impactful_proj']
fig = px.scatter(
    fig_df,
    x=fig_df.timestep,
    y=fig_df.impact_ratio,
    color=fig_df.reward_use_fund.astype(str),
    opacity=1,
    title='impact ratio under different reward_use_fund parameters',
    #trendline="lowess",
    labels={'color': 'Reward parameter'}
)

fig.show()

So even if the reward system is using a bit more funding (assuming reward is given out in the format of $NEAR) and that ratio is more than how much the impactful projects will draw money, the overall ratio of the impactful projects will still increase. If the impact metrics is more focused on projects drawing more money into the NEAR system, as the higher "impactful project to fund" parameters suggests, this will result in a positive reinforcement of more impactful projects in the system.